In [1]:
# 模型构建---D盘

import pandas as pd
inputfile = 'attrsConstruction.xlsx'

data = pd.read_excel(inputfile)
df = data.iloc[:len(data)-5]
df

,SYS_NAME,CWXT_DB:184:C:\,CWXT_DB:184:D:\,COLLECTTIME
0,财务管理系统,34270787.33,80262592.65,2014-10-01
1,财务管理系统,34328899.02,83200151.65,2014-10-02
2,财务管理系统,34327553.50,83208320.00,2014-10-03
3,财务管理系统,34288672.21,83099271.65,2014-10-04
4,财务管理系统,34190978.41,82765171.65,2014-10-05
5,财务管理系统,34187614.43,82522895.00,2014-10-06
6,财务管理系统,34285280.22,82590885.00,2014-10-07
7,财务管理系统,34290578.41,82368173.30,2014-10-08
8,财务管理系统,33211870.40,82172263.30,2014-10-09
9,财务管理系统,33249253.87,81922685.00,2014-10-10


In [2]:
# 第 * 1 * 步--D盘---------平稳性检测
#1)平稳性检测 ：判断是否平稳，若不平稳，对其进行差分处理直至平稳
# 方法：采用单位根检验（ADF）的方法或者时序图的方法（见数据探索模块）
from statsmodels.tsa.stattools import adfuller as ADF
diff = 0
# 判断D盘数据的平稳性，以及确定几次差分后平稳
adf = ADF(df['CWXT_DB:184:D:\\'])
print adf 

while adf[1] >= 0.05 : # adf[1]是p值，p值小于0.05认为是平稳的
    print adf[1]
    diff = diff + 1
    adf = ADF(df['CWXT_DB:184:D:\\'].diff(diff).dropna())#注意，差分后使用ADF检验时，必须去掉空值
    
print (u'原始序列经过%s阶差分后归于平稳，p值为%s') % (diff, adf[1])
df['CWXT_DB:184:D:\\_adf'] = df['CWXT_DB:184:D:\\'].diff(1)


(-2.6460462274943004, 0.083848896341219115, 0L, 41L, {'5%': -2.9351348158036012, '1%': -3.6009833671885199, '10%': -2.6059629803688282}, 958.70113211750595)
0.0838488963412
原始序列经过1阶差分后归于平稳，p值为4.79259126339e-07


D:\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
D:\Anaconda2\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [3]:
# 第 * 2 * 步--D盘---------白噪声检验
# 目的：验证序列中有用信息是否已经被提取完毕，需要进行白噪声检验。若序列是白噪声序列，说明序列中有用信息已经被提取完，只剩随机扰动
# 方法：采用LB统计量的方法进行白噪声检验
# 若没有通过白噪声检验，则需要进行模型识别，识别其模型属于AR、MA还是ARMA。

inputfile2 = 'attrsConstruction.xlsx'
data1 = pd.read_excel(inputfile2)
data1 = data1.iloc[:len(data1)-5]# 不使用最后五个数据（作为预测参考）

# 白噪声检测
from statsmodels.stats.diagnostic import acorr_ljungbox

[[lb], [p]] = acorr_ljungbox(data1['CWXT_DB:184:D:\\'], lags = 1) ## lags是残差延迟个数
if p < 0.05:
    print (u'原始序列为非白噪声序列，对应的p值为：%s' % p)
else:
    print (u'原始序列为白噪声序列，对应的p值为：%s' % p)

[[lb], [p]] = acorr_ljungbox(data1['CWXT_DB:184:D:\\'].diff(1).dropna(), lags = 1)
if p < 0.05:
    print (u'一阶差分序列为非白噪声序列，对应的p值为：%s' % p)
else:
    print (u'一阶差分序列为白噪声序列，对应的p值为：%s' % p)


原始序列为非白噪声序列，对应的p值为：9.95850372977e-06
一阶差分序列为白噪声序列，对应的p值为：0.114330259776


In [4]:
# 第 * 3 * 步--D盘---------模型识别
# 方法：采用极大似然比方法进行模型的参数估计，估计各个参数的值。
# 然后针对各个不同模型，采用BIC信息准则对模型进行定阶，确定p,q参数，从而选择最优模型。
# 注意，进行此步时，index需要为时间序列类型
# 确定最佳p、d、q的值
inputfile3 = 'attrsConstruction.xlsx'
data2 = pd.read_excel(inputfile3,index_col='COLLECTTIME')
xtest_value=data2['CWXT_DB:184:D:\\'][-5:]
data2 = data2.iloc[:len(data2)-5]# 不使用最后五个数据（作为预测参考） 
# print data2
xdata2 = data2['CWXT_DB:184:D:\\']
# print xdata2

from statsmodels.tsa.arima_model import ARIMA#建立ARIMA（p,1，q）模型
# 定阶
# 目前选择模型常用如下准则!!!!!
# 增加自由参数的数目提高了拟合的优良性，
# AIC鼓励数据拟合的优良性但是尽量避免出现过度拟合(Overfitting)的情况。所以优先考虑的模型应是AIC值最小的那一个
#* AIC=-2 ln(L) + 2 k 中文名字：赤池信息量 akaike information criterion (AIC)
# * BIC=-2 ln(L) + ln(n)*k 中文名字：贝叶斯信息量 bayesian information criterion (BIC)
# * HQ=-2 ln(L) + ln(ln(n))*k hannan-quinn criterion (HQ)

pmax = int(len(xdata2)/10) # 一般阶数不超过length/10
qmax = int(len(xdata2)/10) # 一般阶数不超过length/10
bic_matrix = [] # bic矩阵
for p in range(pmax+1):
    tmp = []
    for q in range(qmax+1):
        try:
#             print ARIMA(xdata2, (p,1,q)).fit().bic
            tmp.append(ARIMA(xdata2, (p,1,q)).fit().bic) #存在部分为空值，会报错
        except:
            tmp.append(None)
            
    bic_matrix.append(tmp)
    
bic_matrix = pd.DataFrame(bic_matrix) # 从中可以找出最小值
print bic_matrix
print bic_matrix.stack()


D:\Anaconda2\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
D:\Anaconda2\lib\site-packages\statsmodels\base\model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
D:\Anaconda2\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
D:\Anaconda2\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


             0            1            2            3     4
0  1275.686824  1273.190435  1274.329711  1274.465405  None
1  1276.749128  1272.960083          NaN          NaN  None
2  1279.694296  1277.067761          NaN  1280.092482  None
3  1278.065999  1278.988595  1282.782534  1285.943494  None
4  1281.220791  1282.699992  1286.297519  1290.195037  None
0  0    1275.69
   1    1273.19
   2    1274.33
   3    1274.47
1  0    1276.75
   1    1272.96
2  0    1279.69
   1    1277.07
   3    1280.09
3  0    1278.07
   1    1278.99
   2    1282.78
   3    1285.94
4  0    1281.22
   1     1282.7
   2     1286.3
   3     1290.2
dtype: object


In [5]:
# 第 * 4 * 步--D盘---------模型检验
# 确定模型后，需要检验其残差序列是否是白噪声，若不是，说明，残差中还存在有用的信息，需要修改模型或者进一步提取。
# 若其残差不是白噪声，重新更换p,q的值，重新确定
import pandas as pd
import numpy as np

while 1:
    p, q = bic_matrix.stack().idxmin() # 先展平该表格，然后找出最小值的索引位置
    print (u'当前BIC最小的p值与q值分别为：%s、%s' % (p,q))
    
    lagnum = 12 # 残差延迟个数

    # 由模型识别可知第一次BIC最小的p值与q值分别为：0、1

    arima = ARIMA(xdata2, (p,1,q)).fit() # 建立并训练模型
    xdata_pred = arima.predict(typ = 'levels') # 预测
    pred_error = (xdata_pred - xdata2).dropna() # 计算残差

    # 白噪声检测
    from statsmodels.stats.diagnostic import acorr_ljungbox

    lbx, px = acorr_ljungbox(pred_error, lags = lagnum)
    h = (px < 0.05).sum() # p值小于0.05，认为是非噪声
    if h > 0:
        print (u'模型ARIMA(%s,1,%s)不符合白噪声检验' % (p,q))
        print ('在BIC矩阵中去掉[%s,%s]组合，重新进行计算' % (p,q))
        bic_matrix.iloc[p,q] =  np.nan
        arimafail = arima
        continue
    else:
        print (p,q)
        print (u'模型ARIMA(%s,1,%s)符合白噪声检验' % (p,q))
        break
        


当前BIC最小的p值与q值分别为：1、1
(1, 1)
模型ARIMA(1,1,1)符合白噪声检验


D:\Anaconda2\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [6]:
arima.summary()

D:\Anaconda2\lib\site-packages\statsmodels\tsa\arima_model.py:1441: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(-inv(hess)))
D:\Anaconda2\lib\site-packages\statsmodels\tsa\arima_model.py:1489: RuntimeWarning: invalid value encountered in absolute
  return t.sf(np.abs(self.tvalues), df_resid) * 2
D:\Anaconda2\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
D:\Anaconda2\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
D:\Anaconda2\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                             ARIMA Model Results                              
==============================================================================
Dep. Variable:      D.CWXT_DB:184:D:\   No. Observations:                   41
Model:                 ARIMA(1, 1, 1)   Log Likelihood                -629.053
Method:                       css-mle   S.D. of innovations        1101399.834
Date:                Tue, 14 Nov 2017   AIC                           1266.106
Time:                        21:20:30   BIC                           1272.960
Sample:                    10-02-2014   HQIC                          1268.602
                         - 11-11-2014                                         
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                    1.488e+05        nan        nan        nan         nan         nan
ar.L1.D.CWXT_DB:184:D:\     0.3645      0.055      6.646      0.000       0.257       0.472
ma.L1.D.CWXT_DB:184:D:\    -0.8942        nan        nan        nan         nan         nan
                                    Roots                                    
=============================================================================
                 Real           Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            2.7438           +0.0000j            2.7438            0.0000
MA.1            1.1183           +0.0000j            1.1183            0.0000
-----------------------------------------------------------------------------
"""

In [7]:
forecast_values, forecasts_standard_error, forecast_confidence_interval = arima.forecast(5)
forecast_values

array([ 87643355.43006547,  87697758.83450611,  87812141.77725135,
        87948384.3780629 ,  88092593.7732894 ])

In [8]:
predictdata = pd.DataFrame(xtest_value)
predictdata.insert(1,'CWXT_DB:184:D:\\_predict',forecast_values)
predictdata.rename(columns={'CWXT_DB:184:D:\\':u'实际值','CWXT_DB:184:D:\_predict':u'预测值'},inplace=True)
predictdata.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5 entries, 2014-11-12 to 2014-11-16
Data columns (total 2 columns):
实际值    5 non-null float64
预测值    5 non-null float64
dtypes: float64(2)
memory usage: 120.0 bytes


In [9]:
result_d = predictdata.applymap(lambda x: '%.2f' % x) # 将表格中各个浮点值都格式化
result_d.to_excel('pedictdata_D.xlsx')
result_d

,实际值,预测值
COLLECTTIME,,
2014-11-12,87249335.55,87643355.43
2014-11-13,86986142.20,87697758.83
2014-11-14,86678240.00,87812141.78
2014-11-15,89766600.00,87948384.38
2014-11-16,89377527.25,88092593.77


In [10]:
# 第 * 5 * 步--D盘---------模型评价
# 为了评价时序预测模型效果的好坏，本章采用3个衡量模型预测精度的统计量指标：平均绝对误差、均方根误差、平均绝对百分误差
# -*- coding:utf-8 -*-
import pandas as pd

inputfile4 = 'pedictdata_D.xlsx'
result = pd.read_excel(inputfile4,index_col='COLLECTTIME')
result = result.applymap(lambda x: x/10**6)
print result

# 计算误差
abs_ = (result[u'预测值']-result[u'实际值']).abs()
mae_ = abs_.mean() # mae平均绝对误差
rmas_ = ((abs_**2).mean())**0.5 #rmas均方根误差
mape_ = (abs_/result[u'实际值']).mean() #mape平均绝对百分误差
# print abs_
print mae_
print rmas_
print mape_
errors = 1.5
print '误差阈值为%s' % errors
if (mae_ < errors) & (rmas_ < errors) & (mape_ < errors):
    print (u'平均绝对误差为：%.4f, \n均方根误差为：%.4f, \n平均绝对百分误差为：%.4f' % (mae_, rmas_, mape_))
    print '误差检验通过！'
else:
    print '误差检验不通过！'

                   实际值        预测值
COLLECTTIME                      
2014-11-12   87.249336  87.643355
2014-11-13   86.986142  87.697759
2014-11-14   86.678240  87.812142
2014-11-15   89.766600  87.948384
2014-11-16   89.377527  88.092594
1.068537478
1.17510384473
0.0120819912826
误差阈值为1.5
平均绝对误差为：1.0685, 
均方根误差为：1.1751, 
平均绝对百分误差为：0.0121
误差检验通过！


In [11]:
arimaf = ARIMA(xdata2, (0,1,0)).fit()
# arimaf.forecast(5)[0]
arimaf.summary() #注意当p,q值为0，0时，summary方法报错

IndexError: list index out of range